2. 利用课堂案例，实现分布式DDP模型训练。存盘后加载实现推理。

In [1]:
!pip -q install evaluate seqeval

In [2]:
from transformers import AutoModelForTokenClassification,AutoTokenizer,DataCollatorForTokenClassification
from transformers import TrainingArguments, Trainer
import torch
import evaluate
import seqeval
from datasets import load_dataset


In [3]:
model = AutoModelForTokenClassification.from_pretrained("google-bert/bert-base-chinese",num_labels=7)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

In [5]:
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese")

In [6]:
# 模型测试
message = "命名实体识别"
label = torch.tensor([0, 1, 0, 2, 5, 4])

model_input = tokenizer([message], return_tensors="pt")
result = model(**model_input)
print(result.loss)
print(result.logits)
print(result.logits.shape)  # (batch_size, sequence_length, num_labels)

None
tensor([[[-0.4554, -0.3599,  0.2053, -0.8422,  0.1384,  0.4004,  0.3453],
         [-0.0770, -0.1642,  0.1621, -1.4575,  0.0242,  0.0445,  0.3897],
         [-0.5516, -0.4103, -0.3800, -1.0367,  0.2667, -0.0371, -0.0798],
         [ 0.2075,  0.0971,  0.3932, -1.4048, -0.0305,  0.1208,  0.1933],
         [-0.3234, -0.2060,  0.1865, -1.3074, -0.0425, -0.0970,  0.3460],
         [-0.3628,  0.0504,  0.3074, -1.0486, -0.0066,  0.3090, -0.2814],
         [-0.4035, -0.0642, -0.1623, -0.9808,  0.3029, -0.1835, -0.0783],
         [-0.9058, -0.6511, -0.1494, -0.6673,  0.3933,  0.8644, -0.0998]]],
       grad_fn=<ViewBackward0>)
torch.Size([1, 8, 7])


In [8]:
# 加载 hf中的dataset
ds = load_dataset('nlhappy/CLUE-NER')
ds

ConnectionError: Couldn't reach 'nlhappy/CLUE-NER' on the Hub (LocalEntryNotFoundError)

In [ ]:
# 实体映射数据集词典准备
entities = ["O"] + list(
    {
        "movie",
        "name",
        "game",
        "address",
        "position",
        "company",
        "scene",
        "book",
        "organization",
        "government",
    }
)
tags = ["O"]

for entity in entities:
    tags.append("B-" + entity.upper())
    tags.append("I-" + entity.upper())

entity_index = {entity: i for i, entity in enumerate(entities)}

In [ ]:
entity_index

In [ ]:
tags

In [ ]:
# 实体映射数据集词典准备
entities = ["O"] + list(
    {
        "movie",
        "name",
        "game",
        "address",
        "position",
        "company",
        "scene",
        "book",
        "organization",
        "government",
    }
)
tags = ["O"]

for entity in entities:
    tags.append("B-" + entity.upper())
    tags.append("I-" + entity.upper())

entity_index = {entity: i for i, entity in enumerate(entities)}


def entity_tags_proc(item):
    # item 即是dataset中记录
    text_len = len(item["text"])
    tags = [0] * text_len

    # 遍历实体列表,所有实体类别标记填入tags
    entities = item["ents"]
    for ent in entities:
        indices = ent["indices"]
        label = ent["label"]
        tags[indices[0]] = entity_index[label] * 2 - 1
        for idx in indices[1:]:
            tags[idx] = entity_index[label] * 2

    return {"ent_tag": tags}


ds1 = ds.map(entity_tags_proc)

In [ ]:
ds1

In [ ]:
token_index = tokenizer.encode("2000年2月add", add_special_tokens=False)
print(token_index)
tokens = tokenizer.decode(token_index)
print(tokens)

In [ ]:
input_data = tokenizer(
    [list("2000年2月add")],
    add_special_tokens=False,
    truncation=True,
    is_split_into_words=True,
)
print(input_data)
tokens = tokenizer.decode(token_index)
print(tokens)

In [ ]:
entities = ["O"] + list(
    {
        "movie",
        "name",
        "game",
        "address",
        "position",
        "company",
        "scene",
        "book",
        "organization",
        "government",
    }
)
tags = ["O"]
for entity in entities:
    tags.append("B-" + entity.upper())
    tags.append("I-" + entity.upper())
entity_index = {entity: i for i, entity in enumerate(entities)}


def entity_tags_proc(item):
    text_len = len(item["text"])
    tags = [0] * text_len

    entities = item["ents"]
    for ent in entities:
        indices = ent["indices"]
        label = ent["label"]
        tags[indices[0]] = entity_index[label] * 2 - 1
        for idx in indices[1:]:
            tags[idx] = entity_index[label] * 2

    return {"ent_tag": tags}


ds1 = ds.map(entity_tags_proc)


def data_input_proc(item):
    # 输入文本先拆分为字符,再转换为模型输入的token索引
    batch_texts = [list(text) for text in item["text"]]
    input_data = tokenizer(
        batch_texts,
        truncation=True,
        add_special_tokens=False,
        max_length=512,
        is_split_into_words=True,
        padding="max_length",
    )
    input_data["labels"] = [tag + [0] * (512 - len(tag)) for tag in item["ent_tag"]]
    return input_data


ds2 = ds1.map(data_input_proc, batched=True)

In [ ]:
for item in ds2["train"]:
    print(
        len(item["input_ids"]),
        len(item["token_type_ids"]),
        len(item["attention_mask"]),
        len(item["labels"]),
    )
    break

In [ ]:
# 记录转换为pytorch
ds2.set_format(
    type="torch",
    columns=["input_ids", "token_type_ids", "attention_mask", "labels"],
)

In [ ]:
# 训练模型
[name for name, params in model.named_parameters()]

In [ ]:
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup
import torch.optim as optim

train_dl = DataLoader(
    ds2["train"],
    shuffle=True,
    batch_size=16,
)

id2lbl = {i: tag for i, tag in enumerate(tags)}
lbl2id = {tag: i for i, tag in enumerate(tags)}

model = AutoModelForTokenClassification.from_pretrained(
    "google-bert/bert-base-chinese",
    num_labels=23,
    id2label=id2lbl,
    label2id=lbl2id,
)
model.to("cuda")

# 模型参数分组
param_optimizer = list(model.named_parameters())
bert_params, classifier_params = [], []

for name, param in param_optimizer:
    if "bert" in name:
        bert_params.append(param)
    else:
        classifier_params.append(param)

param_groups = [
    {"params": bert_params, "lr": 1e-5},
    {"params": classifier_params, "weight_decay": 0.1, "lr": 1e-4},
]
optimizer = optim.AdamW(param_groups)
# 学习率调度器
train_steps = len(train_dl) * 5
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=100, num_training_steps=train_steps
)

In [ ]:
from tqdm import tqdm


def train_epoch(model, train_dl, optimizer, scheduler, epoch=5):

    for epoch in range(epoch):
        model.train()
        tpbar = tqdm(train_dl)
        for items in tpbar:
            items = {k: v.to("cuda") for k, v in items.items()}
            optimizer.zero_grad()
            outputs = model(**items)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            scheduler.step()
            tpbar.set_description(
                f"Epoch:{epoch+1}"
                + f"bert_lr:{scheduler.get_lr()[0]}"
                + f"classifier_lr:{scheduler.get_lr()[1]}"
                + f"loss:{loss.item():.4f}"
            )

train_epoch(model, train_dl, optimizer, scheduler, epoch=5)

In [ ]:
# 支持混合精度训练
from tqdm import tqdm
import torch

DEVICE = "cuda"
scaler = torch.GradScaler()


def train_epoch_amp(model, train_dl, optimizer, scheduler, epoch=5):
    for epoch in range(epoch):
        model.train()
        tpbar = tqdm(train_dl)
        for items in tpbar:
            items = {k: v.to(DEVICE) for k, v in items.items()}
            optimizer.zero_grad()

            with torch.autocast(device_type=DEVICE):
                outputs = model(**items)

            loss = outputs.loss
            # 使用混合精度训练
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            scheduler.step()
            tpbar.set_description(
                f"Epoch:{epoch+1}"
                + f"bert_lr:{scheduler.get_lr()[0]}"
                + f"classifier_lr:{scheduler.get_lr()[1]}"
                + f"loss:{loss.item():.4f}"
            )


train_epoch_amp(model, train_dl, optimizer, scheduler, epoch=5)

In [ ]:
%%writefile ddp_simple.py
# 支持分布式训练

import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler

# 设置分布式环境
def setup(rank, world_size):
    os.environ['MASTER_ADDR'] = 'localhost'
    os.environ['MASTER_PORT'] = '12355'
    dist.init_process_group("nccl", rank=rank, world_size=world_size)
    
# 清理分布式环境
def cleanup():
    dist.destroy_process_group()

# 训练函数
def train(rank, world_size):
    setup(rank, world_size)
    
    # 定义模型并将其移动到GPU
    model = models.resnet50().to(rank)
    ddp_model = DDP(model, device_ids=[rank])
    
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss().to(rank)
    optimizer = optim.SGD(ddp_model.parameters(), lr=0.01)
    
    # 数据预处理和加载
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    
    dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
    
    # 分布式训练采样器
    sampler = DistributedSampler(dataset, num_replicas=world_size, rank=rank)
    dataloader = DataLoader(dataset, batch_size=32, sampler=sampler)
    
    scaler = torch.cuda.amp.GradScaler()
    # 训练循环
    for epoch in range(10):
        ddp_model.train()
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(rank), labels.to(rank)
            
            optimizer.zero_grad()
            
            # 混合精度训练
            with torch.cuda.amp.autocast():
                outputs = ddp_model(inputs)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            
            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(ddp_model.parameters(), max_norm=1.0)
            
            scaler.step(optimizer)
            scaler.update()
            
            print(f"Rank {rank}, Epoch [{epoch+1}/10], Loss: {loss.item():.4f}")
    
    cleanup()

def main():
    world_size = torch.cuda.device_count()
    mp.spawn(train, args=(world_size,), nprocs=world_size, join=True)
    
if __name__ == "__main__":
    main()

In [ ]:
!pip -q install evaluate seqeval

In [ ]:
%%writefile ner_ddp.py

import os
import numpy as np
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
)
from transformers import TrainingArguments, Trainer
import torch
import evaluate  # pip install evaluate
import seqeval  # pip install seqeval
from datasets import load_dataset
import torch.distributed as dist
import torch.multiprocessing as mp


# 设置分布式环境
def setup(rank, world_size):
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "12355"
    dist.init_process_group("nccl", rank=rank, world_size=world_size)


# 清理分布式环境
def cleanup():
    dist.destroy_process_group()


def train(rank, world_size):
    setup(rank, world_size)
    # 数据集
    ds = load_dataset("nlhappy/CLUE-NER")
    # entity_index
    entites = ["O"] + list(
        {
            "movie",
            "name",
            "game",
            "address",
            "position",
            "company",
            "scene",
            "book",
            "organization",
            "government",
        }
    )
    tags = ["O"]
    for entity in entites[1:]:
        tags.append("B-" + entity.upper())
        tags.append("I-" + entity.upper())

    entity_index = {entity: i for i, entity in enumerate(entites)}

    tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-chinese")

    def entity_tags_proc(item):
        # item即是dataset中记录
        text_len = len(item["text"])  # 根据文本长度生成tags列表
        tags = [0] * text_len  # 初始值为‘O’
        # 遍历实体列表，所有实体类别标记填入tags
        entites = item["ents"]
        for ent in entites:
            indices = ent["indices"]  # 实体索引
            label = ent["label"]  # 实体名
            tags[indices[0]] = entity_index[label] * 2 - 1
            for idx in indices[1:]:
                tags[idx] = entity_index[label] * 2
        return {"ent_tag": tags}

    # 使用自定义回调函数处理数据集记录
    ds1 = ds.map(entity_tags_proc)

    def data_input_proc(item):
        # 输入文本先拆分为字符，再转换为模型输入的token索引
        batch_texts = [list(text) for text in item["text"]]
        # 导入拆分为字符的文本列表时，需要设置参数is_split_into_words=True
        input_data = tokenizer(
            batch_texts,
            truncation=True,
            add_special_tokens=False,
            max_length=512,
            is_split_into_words=True,
            padding="max_length",
        )
        input_data["labels"] = [tag + [0] * (512 - len(tag)) for tag in item["ent_tag"]]
        return input_data

    ds2 = ds1.map(data_input_proc, batched=True)  # batch_size 1000

    local_rank = rank

    id2lbl = {i: tag for i, tag in enumerate(tags)}
    lbl2id = {tag: i for i, tag in enumerate(tags)}

    model = AutoModelForTokenClassification.from_pretrained(
        "google-bert/bert-base-chinese", num_labels=21, id2label=id2lbl, label2id=lbl2id
    )
    model.to(local_rank)

    args = TrainingArguments(
        output_dir="ner_train",  # 模型训练工作目录（tensorboard，临时模型存盘文件，日志）
        num_train_epochs=3,  # 训练 epoch
        save_safetensors=False,  # 设置False保存文件可以通过torch.load加载
        per_device_train_batch_size=8,  # 训练批次
        per_device_eval_batch_size=8,
        report_to="tensorboard",  # 训练输出记录
        eval_strategy="epoch",
        local_rank=local_rank,  # 当前进程 RANK
        fp16=True,  # 使用混合精度
        lr_scheduler_type="linear",  # 动态学习率
        warmup_steps=100,  # 预热步数
        ddp_find_unused_parameters=False,  # 优化DDP性能
    )

    def compute_metric(result):
        # result 是一个tuple (predicts, labels)

        # 获取评估对象
        seqeval = evaluate.load("seqeval")
        predicts, labels = result
        predicts = np.argmax(predicts, axis=2)

        # 准备评估数据
        predicts = [
            [tags[p] for p, l in zip(ps, ls) if l != -100]
            for ps, ls in zip(predicts, labels)
        ]
        labels = [
            [tags[l] for p, l in zip(ps, ls) if l != -100]
            for ps, ls in zip(predicts, labels)
        ]
        results = seqeval.compute(predictions=predicts, references=labels)

        return results

    data_collator = DataCollatorForTokenClassification(
        tokenizer=tokenizer, padding=True
    )

    trainer = Trainer(
        model,
        args,
        train_dataset=ds2["train"],
        eval_dataset=ds2["validation"],
        data_collator=data_collator,
        compute_metrics=compute_metric,
    )

    trainer.train()


def main():
    world_size = torch.cuda.device_count()
    mp.spawn(train, args=(world_size,), nprocs=world_size, join=True)


if __name__ == "__main__":
    main()

In [ ]:
!python ner_ddp.py